# 1. Evolve only, steady state. 
** task completed and submitted 4/11/18, 6:54pm**

The exact solution can be the Iliescu solution replacing sin(\pi t) with 1. As I told you earlier, if even on the finest mesh the Galerkin solution displays oscillation, increase the viscosity. On the coarser meshes we have to see oscillations. The exact solution can also be another function of your choice, as long as Pe >>1. The convergence plot you get from this point MUST match the convergence plot for the Galerkin method with no stabilization.

## Exact Solution (Iliescu Ex. 4.1)
For each step choose an exact solution and compute the solution for for all the refinement levels. Make sure that there is at least one refinement level on which the standard Galerkin does not show oscillations.



In [1]:
%matplotlib inline 
import matplotlib
from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])
# Exact Solution
c = 16.0 #*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
iliescu = c*h*(0.5+sym.atan(g)/sym.pi)

ue = sym.simplify(iliescu)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = 0 #sym.diff(ue,t) Remove time dependence for now
fe += - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')


In [2]:
# %load main-EFR.py

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

In [3]:
def evolve(nx, u_code, f_code, sigma, mu, velocity):
    degree = 2

    u_exact = Expression(u_code, degree = degree)
    
    f = Expression(f_code, degree = degree)

    mesh = UnitSquareMesh(nx,nx)

    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree)
    def boundary(x, on_boundary):
        return on_boundary
    
    bc = DirichletBC(Q, u_D, boundary)

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_n = Function(Q)
    u_ = Function(Q)

    # Galerkin variational problem
    F = mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)
    bc.apply(A1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)

    # Step 1 Solve on Coarse Grid
    b = assemble(L)
    bc.apply(b)
    solve(A1, u_.vector(), b, 'gmres')
    
    L2, H1 = compute_errors(u_exact, u_, t, mesh)
    maxval, minval = compute_extrema(u_, t)
    
    print(nx,",",L2,",",H1,",",maxval,",",minval)
    
    out_file_uexact = File("results/evolve_steady_u_exact_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    out_file_uexact << ue
    out_file_ubar = File("results/"+str(mu)+"evolve_steady_u_"+str(nx)+".pvd") 
    out_file_ubar << u_

In [4]:
for nx in [25, 50, 100, 200, 400]:
    evolve(nx, u_code, f_code, sigma, mu, b)

25 , 0.575963540234 , 77.7640877026 , 3.33704219468 , -3.00584156147
50 , 0.263090765967 , 66.3351391124 , 2.26109614455 , -1.51168629104
100 , 0.041851765649 , 21.2045691927 , 1.27924732141 , -0.308457376144
200 , 0.00554563623359 , 4.4654137941 , 1.01038153173 , -0.0434018607758
400 , 0.000221173750748 , 0.47857066332 , 0.99200903871 , -0.00095250536929


## Exact Solution
* c = 16.0 #*sym.sin(sym.pi*t)
* h = x*(1-x)*y*(1-y)
* g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
 * iliescu = c*h*(0.5+sym.atan(g)/sym.pi)

nx = [25, 50, 100, 200, 400, 800];


## Problem Parameters
* sigma = 1.0
* b=as_vector([2.0, 3.0])

### Case: mu = 10**(-3)

L2 = [0.00247460360646, 0.000178438487313, 2.73358212482e-05, 3.41629105533e-06, 6.94074779253e-07, 3.6014931185e-07];

H1 = [0.306905633863, 0.0721749666942,  0.0219256336243, 0.00485695915782, 0.00092161831575, 0.000215529295391];

Notes:
* For nx=800 to work, I needed to add 'gmres' to solver options.
* mu = 10**(-3) doesn't produce oscillations in solution from coarse to fine so need to make it smaller.

### Case: mu = 10**(-4)

L2 = [0.060268960767,0.0140569542371,0.00081169760276,5.74359580515e-05,1.06868210769e-05,1.6475746086e-06];

H1 = [8.20787143056,2.98967945006,0.381021312301,0.0973251772743,0.0343154282121,0.00923974756737];

Notes:
* Oscillations appear in nx = 25, 50, 100.


### Case: mu = 10**(-5)

L2 = [0.575963540234,0.263090765967,0.041851765649, 0.00554563623359,0.000221173750748,2.0995474916e-05];

H1 = [77.7640877026,66.3351391124,21.2045691927,4.4654137941,0.47857066332,0.142515173974];

Notes:
* Oscillations appear more dramatically compared to 10**(-4) case. They appear up to nx = 200. By nx = 800, oscillations disappear.
